# Part 0: Single Threaded

In [3]:
import numpy as np
import pandas as pd

# load the diab data set|
from sklearn.datasets import load_diabetes
diab = load_diabetes()

# convert to a Pandas Data Frame
diab_pd = pd.DataFrame(data= np.c_[diab['data'],diab['target']], 
                         columns= np.append(diab['feature_names'], 'target')).sample(frac=1)
print(diab_pd.shape)
diab_pd.head(5)


(442, 11)


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
23,0.045341,0.050680,0.060618,0.031065,0.028702,-0.047347,-0.054446,0.071210,0.133597,0.135612,245.0
341,0.030811,0.050680,0.059541,0.056301,-0.022208,0.001191,-0.032356,-0.002592,-0.024795,-0.017646,263.0
124,-0.005515,-0.044642,0.023973,0.008101,-0.034592,-0.038892,0.022869,-0.039493,-0.015999,-0.013504,121.0
419,-0.020045,-0.044642,-0.054707,-0.053870,-0.066239,-0.057367,0.011824,-0.039493,-0.074093,-0.005220,42.0
271,0.038076,0.050680,0.008883,0.042529,-0.042848,-0.021042,-0.039719,-0.002592,-0.018114,0.007207,127.0


In [5]:
from sklearn.linear_model import LinearRegression
from scipy.stats.stats import pearsonr

# split into data and label arrays 
y = diab_pd['target']
X = diab_pd.drop(['target'], axis=1)

# create training (~80%) and test data sets
X_train = X[:400]
X_test = X[400:]
y_train = y[:400]
y_test = y[400:]

# train a classifier 
lr = LinearRegression()
model = lr.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# error metrics
r = pearsonr(y_pred, y_test)
mae = sum(abs(y_pred - y_test))/len(y_test)
print("R-squared: " + str(r[0]**2))
print("MAE: " + str(mae))


R-squared: 0.44972024389022563
MAE: 43.4355042106305


/var/folders/s9/pkq_ftrs45d7vxwnrztcln6r0000gn/T/ipykernel_36267/3603432283.py:2: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


# Part 1: Native Spark

In [5]:
# convert to a Spark data frame
diab_sp = spark.createDataFrame(diab_pd)
display(diab_sp.take(5))


CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0.03537,34.0,6.09,0.0,0.433,6.59,40.4,5.4917,7.0,329.0,16.1,395.75,9.5,22.0
0.12329,0.0,10.01,0.0,0.547,5.913,92.9,2.3534,6.0,432.0,17.8,394.95,16.21,18.8
23.6482,0.0,18.1,0.0,0.671,6.38,96.2,1.3861,24.0,666.0,20.2,396.9,23.69,13.1
4.42228,0.0,18.1,0.0,0.584,6.003,94.5,2.5403,24.0,666.0,20.2,331.29,21.32,19.1
0.27957,0.0,9.69,0.0,0.585,5.926,42.6,2.3817,6.0,391.0,19.2,396.9,13.59,24.5


In [6]:
from pyspark.ml.feature import VectorAssembler

# split into training and test spark data frames
diab_train = spark.createDataFrame(diab_pd[:400])
diab_test = spark.createDataFrame(diab_pd[400:])

# convert to vector representation for MLlib
assembler = VectorAssembler(inputCols= diab_train.schema.names[:(diab_pd.shape[1] - 1)],  outputCol="features" )
diab_train = assembler.transform(diab_train).select('features', 'target') 
diab_test = assembler.transform(diab_test).select('features', 'target') 

display(diab_train.take(5))

features,target
"List(1, 13, List(), List(0.03537, 34.0, 6.09, 0.0, 0.433, 6.59, 40.4, 5.4917, 7.0, 329.0, 16.1, 395.75, 9.5))",22.0
"List(1, 13, List(), List(0.12329, 0.0, 10.01, 0.0, 0.547, 5.913, 92.9, 2.3534, 6.0, 432.0, 17.8, 394.95, 16.21))",18.8
"List(1, 13, List(), List(23.6482, 0.0, 18.1, 0.0, 0.671, 6.38, 96.2, 1.3861, 24.0, 666.0, 20.2, 396.9, 23.69))",13.1
"List(1, 13, List(), List(4.42228, 0.0, 18.1, 0.0, 0.584, 6.003, 94.5, 2.5403, 24.0, 666.0, 20.2, 331.29, 21.32))",19.1
"List(1, 13, List(), List(0.27957, 0.0, 9.69, 0.0, 0.585, 5.926, 42.6, 2.3817, 6.0, 391.0, 19.2, 396.9, 13.59))",24.5


In [7]:
# linear regresion with Spark
from pyspark.ml.regression import LinearRegression

# linear regression 
lr = LinearRegression(maxIter=10, regParam=0.1, elasticNetParam=0.5, labelCol="target")

# Fit the model
model = lr.fit(diab_train)
diab_pred = model.transform(diab_test)

# calculate results 
r = diab_pred.stat.corr("prediction", "target")
print("R-sqaured: " + str(r**2))


R-sqaured: 0.8063433683277974

In [8]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

crossval = CrossValidator(estimator=LinearRegression(labelCol = "target"),  
                           estimatorParamMaps=ParamGridBuilder().addGrid(LinearRegression.elasticNetParam, [0, 0.5, 1.0]).build(),
                           evaluator=RegressionEvaluator(labelCol = "target", metricName = "r2"),
                           numFolds=10)

# cross validate the model and select the best fit
cvModel = crossval.fit(diab_train) 
model = cvModel.bestModel

# calculate results 
diab_pred = model.transform(diab_test)
r = diab_pred.stat.corr("prediction", "target")
print("R-sqaured: " + str(r**2))


R-sqaured: 0.8148948560944629

# Part 2: Thread Pools

In [10]:
# sklearn version 
from sklearn.ensemble import RandomForestRegressor as RFR
from multiprocessing.pool import ThreadPool

# allow up to 5 concurrent threads
pool = ThreadPool(5)

# hyperparameters to test out (n_trees)
parameters = [ 10, 20, 50]

# define a function to train a RF model and return metrics 
def sklearn_random_forest(trees, X_train, X_test, y_train, y_test):

    # train a random forest regressor with the specified number of trees
    rf= RFR(n_estimators = trees)
    model = rf.fit(X_train, y_train)

    # make predictions
    y_pred = model.predict(X_test)
    r = pearsonr(y_pred, y_test)

    # return the number of trees, and the R value 
    return [trees, r[0]**2]  

# run the tasks 
pool.map(lambda trees: sklearn_random_forest(trees, X_train, X_test, y_train, y_test), parameters)

  

Out[ 7 ]: 
[[10, 0.92121913976894299],
 [20, 0.92413752558900675],
 [50, 0.92705124846648523]]

In [11]:
# spark version
from pyspark.ml.regression import RandomForestRegressor

# define a function to train a RF model and return metrics 
def mllib_random_forest(trees, diab_train, diab_test):

    # train a random forest regressor with the specified number of trees
    rf = RandomForestRegressor(numTrees = trees, labelCol="target")
    model = rf.fit(diab_train)

    # make predictions
    diab_pred = model.transform(diab_test)
    r = diab_pred.stat.corr("prediction", "target")

    # return the number of trees, and the R value 
    return [trees, r**2]
  
# run the tasks 
pool.map(lambda trees: mllib_random_forest(trees, diab_train, diab_test), parameters)
  

Out[ 8 ]: [[10, 0.9237724296567057], [20, 0.9180495321606045], [50, 0.9268564908071572]]

# Part 3: Pandas UDF

In [13]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *

# setup the spark data frame as a table
diab_sp.createOrReplaceTempView("diab")

# add train/test label and expand the data set by 3x (each num trees parameter)
full_df = spark.sql("""
  select *
  from (
    select *, case when rand() < 0.8 then 1 else 0 end as training 
    from diab
  ) b
  cross join (
      select 11 as trees union all select 20 as trees union all select 50 as trees)
""")

schema = StructType([StructField('trees', LongType(), True),
                     StructField('r_squared', DoubleType(), True)])  

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def train_RF(diab_pd):
    trees = diab_pd['trees'].unique()[0]

    # get the train and test groups 
    diab_train = diab_pd[diab_pd['training'] == 1]
    diab_test = diab_pd[diab_pd['training'] == 0] 
        
    # create data and label groups 
    y_train = diab_train['target']
    X_train = diab_train.drop(['target'], axis=1)
    y_test = diab_test['target']
    X_test = diab_test.drop(['target'], axis=1)
   
    # train a classifier 
    rf= RFR(n_estimators = trees)
    model = rf.fit(X_train, y_train)

    # make predictions
    y_pred = model.predict(X_test)
    r = pearsonr(y_pred, y_test)
    
    # return the number of trees, and the R value 
    return pd.DataFrame({'trees': trees, 'r_squared': (r[0]**2)}, index=[0])
  
# use the Pandas UDF
results = full_df.groupby('trees').apply(train_RF)

# print the results 
print(results.take(3))


[Row(trees=20, r_squared=0.8633562691646341), Row(trees=50, r_squared=0.866335129308371), Row(trees=11, r_squared=0.8257884742588874)]